**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Edición 2022**

---
## **Aprendizaje Automático Profundo - Deep Learning**
---

## **Trabajo Práctico Entregable 1 - Red Neuronal Simple**

**Integrantes:** Gastón Briozzo, Tomás Niño Kehoe

**Profesores:** Johanna Frau,  Mauricio Mazuecos

# Importamos las librerias necesarias

In [24]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from sklearn import metrics
from tqdm.notebook import tqdm

import mlflow

import csv
import json
import functools
import gzip
import numpy as np
import pandas as pd
import torch
import tempfile

from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, balanced_accuracy_score

from gensim import corpora
from gensim.models import KeyedVectors
from gensim.parsing import preprocessing
from torch.utils.data import Dataset, DataLoader, IterableDataset
#from torchmetrics import AveragePrecision

from tqdm.notebook import tqdm, trange
import tempfile


In [25]:
torch.__version__

'1.10.1+cu111'

In [26]:
print(f"¿CUDA disponible? {torch.cuda.is_available()}")

¿CUDA disponible? True


## Exploración de datos
Cargamos los datos de entrenamiento para ver que forma tienen

In [27]:
train_dataset = pd.read_json('./data/meli-challenge-2019/spanish.validation.jsonl.gz',lines=True)
#No usar para visualizar, muy pesado. Usar test o val

In [28]:
train_dataset.head(10)

,language,label_quality,title,category,split,tokenized_title,data,target,n_labels,size
0,spanish,unreliable,Metal Biela Dw10 Hdi 2.0,ENGINE_BEARINGS,validation,"[metal, biela, hdi]","[457, 1480, 3450]",88,632,1223820
1,spanish,unreliable,Repuestos Martillo Rotoprcutor Bosch Gshsce Po...,ELECTRIC_DEMOLITION_HAMMERS,validation,"[repuestos, martillo, rotoprcutor, bosch, gshs...","[3119, 892, 1, 767, 1, 9337]",174,632,1223820
2,spanish,unreliable,Pesca Caña Pejerrey Colony Brava 3m Fibra De V...,FISHING_RODS,validation,"[pesca, caña, pejerrey, colony, brava, fibra, ...","[700, 990, 2057, 3990, 3670, 1737, 1153, 6568]",313,632,1223820
3,spanish,unreliable,Porcelanato Abitare Be 20x120 Cm. Ceramica Por...,PORCELAIN_TILES,validation,"[porcelanato, abitare, ceramica, portinari]","[2722, 4404, 1406, 4405]",427,632,1223820
4,spanish,unreliable,Reconstruction Semi Di Lino Alfaparf Shampoo 1...,HAIR_SHAMPOOS_AND_CONDITIONERS,validation,"[reconstruction, semi, lino, alfaparf, shampoo]","[1, 3365, 7502, 10919, 849]",194,632,1223820
5,spanish,unreliable,"Mascara Fotosensible Lüsqtoff, Oferta Y En Cuo...",WELDING_MASKS,validation,"[mascara, fotosensible, lüsqtoff, oferta, cuotas]","[846, 2521, 38384, 446, 769]",407,632,1223820
6,spanish,unreliable,Bermuda John Cena 14/16 Original,SHORTS,validation,"[bermuda, john, cena, original]","[998, 2168, 5925, 188]",198,632,1223820
7,spanish,unreliable,20x Rueda Neumático Tuerca Set De 20 Lr068126 ...,MOTORCYCLE_WHEEL_AXLES,validation,"[rueda, neumático, tuerca, set, oem, tierr]","[458, 1317, 11808, 489, 859, 1]",578,632,1223820
8,spanish,unreliable,Pelota De Basquet Spalding Tf-elite Nº 6,BASKET_BALLS,validation,"[pelota, basquet, spalding, elite]","[1211, 953, 6649, 3377]",187,632,1223820
9,spanish,unreliable,Placard De Algarrobo Original 3 Puertas,WARDROBES,validation,"[placard, algarrobo, original, puertas]","[1537, 3898, 188, 3616]",292,632,1223820


In [29]:
#train_dataset.data.value_counts()

In [30]:
#train_dataset.label_quality.value_counts()

## Dataset

Tomado de https://github.com/DiploDatos/AprendizajeProfundo/blob/master/3_datasets.ipynb


### Esta clase codifica el Target. 
Éste pasa de ser una variable unidimensional discretizada con valores de 1 a 632 
a una variable con 632 dimensiónes, cada una con valor 0 o 1.

Esta transformación es necesaria para la posterior implementación de la red neuronal,
dado que si el target se presenta como un valor ordinal, la red interpretará que este
orden tiene relevancia. 
Para evitar esto, cada categoría en el target debe tener su propia columna.

In [31]:
class MeLiChallengeDataset(IterableDataset):
    def __init__(self, path, transform=None, key = 'data'):
        """
        path: Ubicación a los datos (comprimidos con gzip)
        key: Columna que vamos a usar para entrenar
        """
        self.dataset_path = path
        self.transform = transform
        self.key = key

    def __iter__(self):
        """
        Habilita un iterador sobre los datos
        """
        with gzip.open(self.dataset_path, "rt") as fh:
            for l in fh:
                data = json.loads(l)    
                
                # one-hot encoding like.
                index = int(data['target'])
                encoded_target = np.zeros(632)
                encoded_target[index] = 1

                item = {
                    "data": data[self.key],
#                     "target": data['target']
                    "target": encoded_target #data['target']
                }
                
                if self.transform:
                    yield self.transform(item)
                else:
                    yield item

### Preprocesamiento  (NO NECESARIO SI USAMOS DATA)
Este se encargará de preprocesar y normalizar el texto. Las palabras se transformarán  en índices de un diccionario, se creará un vector con la secuencia de palabras y se lo pasará a la matriz de embeddings.

In [32]:
#class RawDataProcessor:
#    def __init__(self, 
#                 dataset, 
#                 ignore_header=True, 
#                 filters=None, 
#                 vocab_size=50001):
#        if filters:
#            self.filters = filters
#        else:
#            self.filters = [
#                lambda s: s.lower(),
#                preprocessing.strip_tags,
#                preprocessing.strip_punctuation,
#                preprocessing.strip_multiple_whitespaces,
#                preprocessing.strip_numeric,
#                preprocessing.remove_stopwords,
#                preprocessing.strip_short,
#            ]
#        
#        # Create dictionary based on all the reviews (with corresponding preprocessing)
#        # https://radimrehurek.com/gensim/corpora/dictionary.html
#        self.dictionary = corpora.Dictionary(
#            dataset["tokenized_title"].map(self._preprocess_string).tolist()
#        )
#        # Filter the dictionary with extremos words
#        # https://tedboy.github.io/nlps/generated/generated/gensim.corpora.Dictionary.filter_extremes.html?highlight=filter_extrem
#        self.dictionary.filter_extremes(no_below=2, no_above=1, keep_n=vocab_size)
#        
#        # Make the indices continuous after some words have been removed
#        # https://tedboy.github.io/nlps/generated/generated/gensim.corpora.Dictionary.compactify.html
#        self.dictionary.compactify()
#        
#        # Add a couple of special tokens
#        self.dictionary.patch_with_special_tokens({
#            "[PAD]": 0,
#            "[UNK]": 1
#        })
#        self.idx_to_target = sorted(dataset["target"].unique())
#        self.target_to_idx = {t: i for i, t in enumerate(self.idx_to_target)}
#
#
#    def _preprocess_string(self, string):
#        # https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.preprocess_string:~:text=gensim.parsing.preprocessing.preprocess_string
#        return preprocessing.preprocess_string(string, filters=self.filters)
#
#    def _sentence_to_indices(self, sentence):
#      # https://radimrehurek.com/gensim/corpora/dictionary.html#:~:text=doc2idx(document,via%20unknown_word_index.
#        return self.dictionary.doc2idx(sentence, unknown_word_index=1)
#    
#    def encode_data(self, data):
#        return self._sentence_to_indices(self._preprocess_string(data))
#    
#    def encode_target(self, target):
#        return self.target_to_idx[target]
#    
#    def __call__(self, item):
#        if isinstance(item["data"], str):
#            data = self.encode_data(item["data"])
#        else:
#            data = [self.encode_data(d) for d in item["data"]]
#        
#        if isinstance(item["target"], str):
#            target = self.encode_target(item["target"])
#        else:
#            target = [self.encode_target(t) for t in item["target"]]
#        
#        return {
#            "data": data,
#            "target": target
#        }

### Lectura de datos (NO EJECUTAR SI USAMOS DATA)
Este se encargará de preprocesar y normalizar el texto. Las palabras se transformarán  en índices de un diccionario, se creará un vector con la secuencia de palabras y se lo pasará a la matriz de embeddings.

In [33]:
#dataset = pd.read_json('./data/meli-challenge-2019/spanish.train.jsonl.gz',lines=True)

#preprocess = RawDataProcessor(dataset)

#train_indices, test_indices = train_test_split(dataset.index, test_size=0.2, random_state=42)

#train_dataset = IMDBReviewsDataset(dataset.loc[train_indices].reset_index(drop=True), transform=preprocess)

#test_dataset = IMDBReviewsDataset(dataset.loc[test_indices].reset_index(drop=True), transform=preprocess)

#print(f"Datasets loaded with {len(train_dataset)} training elements and {len(test_dataset)} test elements")
#print(f"Sample train element:\n{train_dataset[0]}")

### Esta clase estandariza las entradas. 
Las entradas son vectores cuya dimensión se corresponde con la cantidad de palabras empleadas para describir el artículo. Los valores en cada dimensión vienen dados por la palabra correspondiente.
Esta clase añade dimensiones al vector de entrada de modo que todos tengan la misma dimensionalidad (20).

Esta transformación es necesaria para la posterior implementación de la red neuronal,
dado que estas requieren que las entradas que alimentarán el modelo tengan la misma dimensionalidad.

In [34]:
class PadSequences:
    def __init__(self, pad_value=0, max_length=None, min_length=1):
        assert max_length is None or min_length <= max_length
        self.pad_value = pad_value
        self.max_length = max_length
        self.min_length = min_length

    def __call__(self, items):
        data, target = list(zip(*[(item["data"], item["target"]) for item in items]))
        seq_lengths = [len(d) for d in data]
        
        if self.max_length:
            max_length = self.max_length
            seq_lengths = [min(self.max_length, l) for l in seq_lengths]
        else:
            max_length = max(self.min_length, max(seq_lengths))

        data = [d[:l] + [self.pad_value] * (max_length - l)
                for d, l in zip(data, seq_lengths)]
            
        return {
           "data": torch.LongTensor(data),
#             "data": torch.FloatTensor(data), 
            "target": torch.FloatTensor(target)
            
        }

In [35]:
# Definimos una collate_fn que nos retorne todos los arreglos de la misma longitud
# data_len = train_dataset.data.apply(lambda v: len(v))
# max_len = data_len.max()


pad_to_len = PadSequences(max_length=20)

### Juntando todo
Tenemos
* Una clase con la responsabilidad de entregar datos, potencialmente preprocesandolos si hace falta
* Una función que transforma los datos leidos para que todos los valores tengan la misma longitud

A partir de esto, creamos dos instancia del `DataLoader`: Uno para cargar los datos de entrenamiento y otro para cargar los datos de _test_

In [36]:
train_dataset = MeLiChallengeDataset('./data/meli-challenge-2019/spanish.train.jsonl.gz', key = 'data')
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=False, num_workers=0, collate_fn = pad_to_len)

In [37]:
test_dataset = MeLiChallengeDataset('./data/meli-challenge-2019/spanish.validation.jsonl.gz', key = 'data') 
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=0, collate_fn = pad_to_len)

## MLP

Tomamos el MLP de las clases y agregamos los parametros que nos interesan

* Capa de entrada: Tantas neuronas como tokens
* Capa de salida: Tantas neuronas como categorías (632)

In [38]:
# if 'casita' in dictionary:
#     print('bien')

In [46]:
import bz2

In [70]:
class MLP(nn.Module):
    def __init__(self, input_size, output_size, hidden, 
                 pretrained_embeddings_path = "./data/SBW-vectors-300-min5.txt.bz2", 
                 dictionary_path            = "./data/meli-challenge-2019/spanish_token_to_index.json.gz",#preprocess.dictionary,
                 freeze_embedings           = True):
        """
        input_size: Número de neuronas de entrada
        output_size: Número de neuronas de salida
        hidden: Lista con los numeros de capas ocultas
        """
        super().__init__()
        assert len(hidden) > 0
        
        self._name = str(input_size) +'_'+ "_".join(map(lambda i: str(i), hidden)) + '_' + str(output_size)
        
        f = gzip.open(dictionary_path, "rt")
        raw_json = f.read()
        dictionary = json.loads(raw_json)
        dictionary
        
        embeddings_matrix = torch.randn(len(dictionary), input_size)
        embeddings_matrix[0] = torch.zeros(input_size)
        
        with bz2.open(pretrained_embeddings_path, "rt") as fh:
            for line in fh:
                word, vector = line.strip().split(None, 1)
                
                if word in dictionary:
                    embeddings_matrix[dictionary[word]] =\
                        torch.FloatTensor([float(n) for n in vector.split()])
                    
        self.embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                                       freeze=freeze_embedings,
                                                       padding_idx=0)
        
        ## Set up layers
        neurons = [input_size]  + hidden 
        parts = []
        for idx, each in enumerate(neurons[:-1]):
            parts.append(nn.Linear(each, neurons[idx + 1]))
            parts.append(nn.ReLU())
            
        parts = parts + [nn.Linear(neurons[-1], output_size), nn.Sigmoid()]
        
        self.model = nn.Sequential(*parts)
    
    def forward(self, x: torch.Tensor):
        x = self.embeddings(x)
        x = torch.mean(x, dim=1)
        x = self.model(x)
        return x
    
    def name(self):
        return "MLP_" + self._name
    

In [40]:
## ESTA CELDA NO VA; EXTRAER EMBEDDING DE AQUI PARA CELDA ANTERIOR
#class IMDBReviewsClassifier(nn.Module):
#    def __init__(self, 
#                 pretrained_embeddings_path, 
#                 dictionary,
#                 vector_size,
#                 freeze_embedings):
#        super().__init__()
#        embeddings_matrix = torch.randn(len(dictionary), vector_size)
#        embeddings_matrix[0] = torch.zeros(vector_size)
#        with gzip.open(pretrained_embeddings_path, "rt") as fh:
#            for line in fh:
#                word, vector = line.strip().split(None, 1)
#                if word in dictionary.token2id:
#                    embeddings_matrix[dictionary.token2id[word]] =\
#                        torch.FloatTensor([float(n) for n in vector.split()])
#        self.embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
#                                                       freeze=freeze_embedings,
#                                                       padding_idx=0)
#        self.hidden1 = nn.Linear(vector_size, 128)
#        self.hidden2 = nn.Linear(128, 128)
#        self.output = nn.Linear(128, 1)
#        self.vector_size = vector_size
#    
#    def forward(self, x):
#        x = self.embeddings(x)
#        x = torch.mean(x, dim=1)
#        x = F.relu(self.hidden1(x))
#        x = F.relu(self.hidden2(x))
#        x = torch.sigmoid(self.output(x))
#        return x

In [71]:
MLP(300, 2 , hidden = [16], pretrained_embeddings_path = "./data/SBW-vectors-300-min5.txt.bz2" ).name()

KeyboardInterrupt: 

In [72]:
MLP(300,632,[256])

MLP(
  (embeddings): Embedding(50002, 300, padding_idx=0)
  (model): Sequential(
    (0): Linear(in_features=300, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=632, bias=True)
    (3): Sigmoid()
  )
)

### Armado paso a paso

In [74]:
# Funciones utilitarias para obtener los datos en batch
get_train_dataloader = lambda bs: DataLoader(train_dataset, batch_size=bs, shuffle=False, num_workers=0, collate_fn = pad_to_len)
get_test_dataloader = lambda bs: DataLoader(test_dataset, batch_size=bs, shuffle=False, num_workers=0, collate_fn = pad_to_len)

#### Verificamos la forma de la entrada, salida y la etiqueta

In [75]:
it = iter(train_dataloader)
e = it.next()
data, target = e['data'], e['target']
data, target

(tensor([[50001,     2, 50000,  ...,     0,     0,     0],
         [    6,     4,     5,  ...,     0,     0,     0],
         [    9,     7,    10,  ...,     0,     0,     0],
         ...,
         [  532,   398,   530,  ...,     0,     0,     0],
         [  534,   536,     1,  ...,     0,     0,     0],
         [  370,    76,   539,  ...,     0,     0,     0]]),
 tensor([[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]))

In [76]:
data.size(), target.size() # batch_size = 4

(torch.Size([128, 20]), torch.Size([128, 632]))

In [77]:
data

tensor([[50001,     2, 50000,  ...,     0,     0,     0],
        [    6,     4,     5,  ...,     0,     0,     0],
        [    9,     7,    10,  ...,     0,     0,     0],
        ...,
        [  532,   398,   530,  ...,     0,     0,     0],
        [  534,   536,     1,  ...,     0,     0,     0],
        [  370,    76,   539,  ...,     0,     0,     0]])

In [78]:
m = MLP(300, 632,[256])
output = m(data)
output, target

(tensor([[0.5081, 0.5064, 0.5086,  ..., 0.4846, 0.5042, 0.5128],
         [0.5085, 0.5062, 0.5091,  ..., 0.4853, 0.5048, 0.5119],
         [0.5097, 0.5060, 0.5085,  ..., 0.4854, 0.5053, 0.5119],
         ...,
         [0.5087, 0.5063, 0.5086,  ..., 0.4855, 0.5046, 0.5121],
         [0.5098, 0.5063, 0.5105,  ..., 0.4869, 0.5051, 0.5121],
         [0.5095, 0.5062, 0.5084,  ..., 0.4861, 0.5051, 0.5121]],
        grad_fn=<SigmoidBackward0>),
 tensor([[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]))

In [79]:
# Convertimos a la forma esperada
#cmp = lambda t: torch.Tensor([[x] for x in t])
cmp = lambda t: t #ID
actual = cmp(target)
actual

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

#### Verificamos que se pueda calcular la función de pérdida y el _score_

In [80]:
loss = nn.BCELoss()
loss_value = loss(output, actual)
loss_value

tensor(0.6923, grad_fn=<BinaryCrossEntropyBackward0>)

In [81]:
categories_from_tensor = lambda t: torch.max(t.data, 1)[1]
y_true = categories_from_tensor(target)
y_true.numpy()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  21,
        38,  39,  40,  41,  42,  43,  29,  44,  45,  46,  47,  48,  49,
        50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,
        45,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  71,
        74,  75,  76,  77,  78,  76,  79,  80,  81,  27,  82,  83,  39,
        84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,
        79,  97,  98,  34,  99,  90,  91, 100, 101, 102, 103, 104, 105,
       106, 107,  86,  78,  77, 108, 109, 110,  85,  26, 111])

In [82]:
predicted = categories_from_tensor(output)
predicted.numpy(), target.numpy() 

(array([369, 369, 369, 531,  37,  48, 369,  48, 531, 369, 156, 496, 369,
        369, 531, 531, 531,  48, 531, 531, 520, 531, 531,  48, 531,  17,
        369, 604, 189, 369, 531, 531, 369, 369, 369, 531, 531, 604, 369,
        531, 531,  57, 369, 531, 496, 531, 369,  48, 531,  48, 520, 369,
        531, 189,  57, 369, 369, 491, 531, 520, 531, 369, 369, 461, 531,
        534, 531, 520, 531, 369, 531, 531, 369, 381, 604, 369, 369, 369,
        369, 369, 369, 369, 369, 105, 531, 531, 369, 531, 604, 369, 531,
         57,  48, 604,  48, 369, 369, 369, 369, 531, 369, 369,  48, 369,
         48, 531, 531,  72, 381, 369, 520, 531, 531, 369, 531, 369, 491,
        531, 520, 520, 369, 369, 369,  57, 461, 369, 531, 531]),
 array([[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32))

In [83]:
balanced_accuracy_score(categories_from_tensor(target).numpy(), predicted.numpy())

/users/tnino/anaconda3/envs/deeplearning/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


0.0

#### Loop de entrenamiento

In [84]:
def run_train(dataloader, model, optimizer, loss):
    assert model.training, "model should be in training mode"
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Run train loop in ${device}")
    running_loss = []
    model.to(device)
    for idx, batch in enumerate(tqdm(dataloader)):
        optimizer.zero_grad()

        input_data = batch["data"].to(device)
        output = model(input_data)

        target_data = batch["target"].to(device)
        
        target = cmp(target_data).to(device)
                            
        loss_value = loss(output, target)
        loss_value.backward()

        optimizer.step()
        running_loss.append(loss_value.item())
    return running_loss

In [85]:
optimizer = optim.Adam(m.parameters(), lr=1e-3, weight_decay=1e-5)
loss = nn.BCELoss()
m.train()
for epoch in trange(1):
    run_train(test_dataloader, m, optimizer, loss)

  0%|          | 0/1 [00:00<?, ?it/s]

Run train loop in $cuda:0


0it [00:00, ?it/s]

#### Loop de validación

In [44]:
def run_eval(dataloader, model, loss):
    assert not model.training, "model should not be in training mode"
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Run eval loop in ${device}")
    running_loss = []
    targets = []
    predictions = []
                     
    model.to(device)
    for batch in tqdm(dataloader):
        input_data = batch["data"].to(device)
        output = model(input_data)
                    
        target = batch["target"]
        target_t = cmp(target).to(device)
        
        running_loss.append(
            loss(output, target_t).item()
        )
        
        predicted = categories_from_tensor(output)
        y_true = categories_from_tensor(target)

        targets.extend(y_true.cpu().numpy())
        predictions.extend(predicted.cpu().numpy())
        
    return running_loss, targets, predictions


In [ ]:
# loss = nn.BCELoss()
# m.eval()

# for epoch in trange(1):
#     _, targets, predictions = run_eval(test_dataloader, m, loss)
#     print(balanced_accuracy_score(targets, predictions))


---

In [45]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [46]:
def run_experiment(params, **kwargs):
    """
    Run a single experiment

    params: A list of (model, loss_fn, optimizer,num_epochs) tuples
    """
    for(build_model, build_loss, build_optimizer, nepochs, batch_sizes) in params:

        # Pasamos varios valores para epochs, ejecutamos un experimento para cada uno
        for epochs in nepochs:
            for bs in batch_sizes:
                
                print(f"Experiment epochs:{epochs}, batch-size:{bs}")
                train_dataloader = get_train_dataloader(bs)
                test_dataloader = get_test_dataloader(bs)
                

                with mlflow.start_run():
                    # Crea nuevos objetos para cada epoca
                    model = build_model()
                    optimizer = build_optimizer(model)
                    loss = build_loss()

                    exp_name = model.name() + "_e" + str(epochs) + "_b" + str(bs)
                    mlflow.set_experiment(exp_name)

                    mlflow.log_param("model_name", model.name())
                    mlflow.log_param("epochs", str(epochs))
                    mlflow.log_param("batch_size", str(bs))

                    model.to(device)
                    for epoch in trange(epochs):
                        model.train()
                        running_loss = run_train(train_dataloader, model, optimizer, loss)
                        mlflow.log_metric("train_loss", sum(running_loss) / len(running_loss), epoch)

                        model.eval()
                        running_loss, targets, predictions = run_eval(test_dataloader, model, loss)
                        mlflow.log_metric("test_loss", sum(running_loss) / len(running_loss), epoch)
                        mlflow.log_metric("test_avp", balanced_accuracy_score(targets, predictions), epoch)

                    print(f"Save results. epochs:{epochs}, batch-size:{bs}")
                    with tempfile.TemporaryDirectory() as tmpdirname:
                        targets = []
                        predictions = []

                        for batch in tqdm(test_dataloader):
                            output = model(batch["data"].to(device))
                            
                            y_true = categories_from_tensor(batch["target"])
                            targets.extend(y_true.numpy())
                            
                            predicted = categories_from_tensor(output)
                            predictions.extend(predicted.squeeze().detach().cpu().numpy())

                        filename = "{}/{}_predictions.csv.gz".format(tmpdirname, exp_name)
                        pd.DataFrame({"prediction": predictions, "target": targets}).to_csv(
                            filename, index=False
                        )
                        mlflow.log_artifact(filename)

In [47]:
output_size = 632

In [48]:
loss = lambda : nn.BCELoss()

def getOptimizer(model, lr = 1e-3, wd = 1e-5):
    return optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
                   
epochs = [1, 2, 4]
batch_sizes = [24, 48, 64]

In [49]:
mlp_test_experiment = [(lambda : MLP(20, output_size, hidden = [256]), loss, getOptimizer, [1], [4])]
#run_experiment(mlp_test_experiment)

In [ ]:


mlp_single_layer = [(lambda : MLP(20, output_size, hidden = [256]), loss, getOptimizer, epochs, batch_sizes)]
mlp_multiple_layers = [(lambda : MLP(20, output_size, hidden = [40,64,256, 512]), loss, getOptimizer, [20], [15,30])]

In [ ]:
#run_experiment(mlp_single_layer)

In [ ]:
#run_experiment(mlp_multiple_layers)

In [50]:
run_experiment([(lambda : MLP(50, output_size, hidden = [1024]), loss, getOptimizer, [20], [160])])

Experiment epochs:20, batch-size:160


2022/11/13 13:34:49 INFO mlflow.tracking.fluent: Experiment with name 'MLP_50_1024_632_e20_b160' does not exist. Creating a new experiment.


  0%|          | 0/20 [00:00<?, ?it/s]

Run train loop in $cuda:0


0it [00:00, ?it/s]

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)